In [1]:
import sys
sys.path.append('../..')
import src.data.data_loader as dl
from src.features.vectorizer import Vectorizer
from src.models.topic_models import TopicModel
import pandas as pd
import sklearn.utils as skutil
pd.set_option('display.max_rows', None)

C:\Users\Maria\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Data 
language = 'english'
typex = 'forum'

# Vectorization
min_df = 0.005
max_df = 0.9

# Topic Modeling
algorithm = 'lda'
num_topics = 110

train_percentage = 0.9
alpha = 'auto'
eta = 0.01
iterations = 200
passes = 60
chunksize = 5000
kappa = 0
tau_0 = 0

In [3]:
data = dl.get_forum_threads_by_language(language, typex,kind = "tagged")
texts = data['thread_texts']

def min_length (texts,min_characters):
    neu = []
    for t in texts: 
        token_perdoc_list = t.split()
        token_min_character = []
        for token in token_perdoc_list: 
            if len(token)>= min_characters: 
                token_min_character.append(token)
        joined = (" ").join(token_min_character)
        neu.append(joined)
    return neu

texts = min_length(texts,3)

KeyError: 'article_title_tokenized'

In [ ]:
vec = Vectorizer('tf', texts, min_df=min_df, max_df=max_df)
vec.save('tagged/vectorizer/{}_{}_{}_{}_pos.pkl'.format(algorithm, language, typex,"forum","tagged"))

document_term_matrix = vec.get_document_token_matrix(texts)
id2token = vec.get_id2token_mapping()

In [ ]:
document_term_matrix = skutil.shuffle(document_term_matrix, random_state=1)
num_docs, num_terms = document_term_matrix.shape
train = int(num_docs * train_percentage)
train_document_term_matrix = document_term_matrix[0:train, :]
test_document_term_matrix = document_term_matrix[train: num_docs, :]

In [ ]:
model = TopicModel('lda', num_topics, train_document_term_matrix, id2token, alpha=alpha, eta=eta, iterations=iterations, passes=passes, chunksize=chunksize, test_document_term_matrix=test_document_term_matrix, decay=kappa, offset=tau_0)
model.save('tagged/topic_models/lda/{}_{}_{}_{}forum{}.pkl'.format(algorithm, language, typex,"tagged",num_topics))